In [10]:
import numpy as np
np.random.seed(13)
from sklearn import svm
import pandas as pd

import math
from __future__ import division
from sklearn.model_selection import KFold

In [11]:
ticker='ADBE'

In [12]:
sentimentVector =  pd.read_csv('SentimentNews/'+ticker+'.csv')
price= pd.read_csv('/home/andrea/Desktop/NLFF/DataSetIndexesLabeled/indexes'+ticker+'.csv')
#print(price)

In [13]:
#alignment of data
maxdata=max(sentimentVector['initTime'])
mindata=min(sentimentVector['initTime'])
price=price[price['Unnamed: 0']>=mindata]
price=price[price['Unnamed: 0']<=maxdata]



In [14]:
#remove the time from data
sentimentVector=sentimentVector.drop(['Unnamed: 0', 'initTime' ], axis=1)

In [15]:

#MEAN on the word count
#print(sentimentVector)
sentimentVector=pd.rolling_mean(sentimentVector,30)
sentimentVector=sentimentVector.iloc[30:]
price=price.iloc[30:]
#print(sentimentVector)


/home/andrea/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: pd.rolling_mean is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(window=30,center=False).mean()
  after removing the cwd from sys.path.


In [ ]:
''''
#MOMENTUM FUTURE 
#wuit htis momentum i'm trying to predict a trend of momentum days
momentum=30
y=[]
x=[]
for i in range(1,len(price)-momentum):
    
    y.append(np.sign(price.iloc[i+momentum]['close']-price.iloc[i]['open']))
    x.append(sentimentVector.iloc[i+momentum].values)
x=np.array(x)
y=np.array(y)
print(len(x))
print(len(y))
''''

In [ ]:
#1
#label because of the maket and append values without data
windowpast=5
x=[]
y=[]
for i in range(windowpast,len(price)):
    #before change from 2 to price-1
    y.append(np.sign(price.iloc[i]['close']-price.iloc[i-windowpast]['close']))
    #after change from 1 to price
    x.append(sentimentVector.iloc[i].values)
x=np.array(x)
y=np.array(y)
print(len(x))
print(len(y))

In [16]:
#2
#label because of the market and append values without datatime
#Important to set lenth of trend in this to see if news affect from today to today+tendwindowtime
#this is theorical (looks at difference betwwen price before news and price after news)because for make it real i have to change price.iloc[i-1]['open'] to price.iloc[i]['open']
#because the trader can buy only on price.iloc[i]['open'] that is today
trendwindowtime=30
x=[]
y=[]
for i in range(1,len(price)-trendwindowtime):
    
    y.append(np.sign(price.iloc[i+trendwindowtime]['close']-price.iloc[i-1]['open']))
    x.append(sentimentVector.iloc[i].values)
x=np.array(x)
y=np.array(y)
print(len(x))
print(len(y))

1658
1658


In [ ]:
#3
#label because of the market and append values without datatime
#Important to set lenth of trend in this to see if news affect from today to today+tendwindowtime
#this is pratical what a trader can really do
trendwindowtime=10
x=[]
y=[]
for i in range(1,len(price)-trendwindowtime):
    
    y.append(np.sign(price.iloc[i+trendwindowtime]['close']-price.iloc[i]['open']))
    x.append(sentimentVector.iloc[i].values)
x=np.array(x)
y=np.array(y)
print(len(x))
print(len(y))

In [ ]:
''''
#MOMENTUM PAST I THINK IS WRONG
momentum=30
y=[]
x=[]
for i in range(momentum+1,len(price)):
    
    y.append(np.sign(price.iloc[i]['close']-price.iloc[i-momentum]['close']))
    x.append(sentimentVector.iloc[i].values)
x=np.array(x)
y=np.array(y)
print(y)
print(x)
''''

In [17]:
#Split betwwen train-validation and test
train=0.8
nt=math.ceil(len(x)*train)
x_tv=[]
y_tv=[]
x_test=[]
y_test=[]
x_tv=x[:nt]
y_tv=y[:nt]
x_test=x[nt:]
y_test=y[nt:]
print(len(x_tv))
print(len(x_test))

1327
331


In [18]:
#Fairly sampling the test 50% 50%
posindex=np.where( y_test > 0 )
negindex=np.where( y_test < 0 )
print(len(posindex[0]))
print(len(negindex[0]))
yindex=[]
nindex=min(len(posindex[0]),len(negindex[0]))
print(nindex)
#for i in range(1,nindex):
y_testnew=np.concatenate((y_test[posindex[0][0:nindex]],y_test[negindex[0][0:nindex]]))
x_testnew=np.concatenate((x_test[posindex[0][0:nindex]],x_test[negindex[0][0:nindex]]))
print(len(y_testnew))
print(len(x_testnew))

184
147
147
294
294


In [19]:
#Find the best gamma and c run only ones

nfold=10
kf = KFold(n_splits=nfold, random_state=13, shuffle=True)


cspace=np.logspace(-3,4,30)
gspace=np.logspace(-6,4,30)
bestsvm=None
maxacc=0
cvacc=0
maxg=0
maxc=0
#model selection
for c in cspace:
    for g in gspace:
        print('-------------Gamma e C---------------:',g,c)
        cvacc=0
        #crossvalidation
        for train_index, test_index in kf.split(x_tv):
            x_train, x_val = x_tv[train_index], x_tv[test_index]
            y_train, y_val = y_tv[train_index], y_tv[test_index]
            rbf_svm=svm.SVC(kernel='rbf',C=c,gamma=g)
            rbf_svm.fit(x_train,y_train)
            accuracy=rbf_svm.score(x_val,y_val)
            cvacc=cvacc+accuracy/nfold
        print('CVacc:',cvacc)
        if(cvacc>maxacc):
            maxacc=cvacc
            maxg=g
            maxc=c
            bestsvm=rbf_svm
print('')
print('ACCURACY:',maxacc)
print('Gamma:',maxg)
print('C:',maxc)
print(bestsvm)
testacc=bestsvm.score(x_testnew,y_testnew)
print('Accuracy on Test set:',testacc)


-------------Gamma e C---------------: 1e-06 0.001
CVacc: 0.6164957849168375
-------------Gamma e C---------------: 2.2122162910704504e-06 0.001
CVacc: 0.6164957849168375
-------------Gamma e C---------------: 4.893900918477499e-06 0.001
CVacc: 0.6164957849168375
-------------Gamma e C---------------: 1.082636733874054e-05 0.001
CVacc: 0.6164957849168375
-------------Gamma e C---------------: 2.395026619987486e-05 0.001
CVacc: 0.6164957849168375
-------------Gamma e C---------------: 5.2983169062837125e-05 0.001
CVacc: 0.6164957849168375
-------------Gamma e C---------------: 0.00011721022975334806 0.001
CVacc: 0.6164957849168375
-------------Gamma e C---------------: 0.0002592943797404667 0.001
CVacc: 0.6164957849168375
-------------Gamma e C---------------: 0.0005736152510448681 0.001
CVacc: 0.6164957849168375
-------------Gamma e C---------------: 0.0012689610031679235 0.001
CVacc: 0.6164957849168375
-------------Gamma e C---------------: 0.0028072162039411786 0.001
CVacc: 0.6164957

CVacc: 0.6164957849168375
-------------Gamma e C---------------: 38.56620421163472 0.0030391953823131978
CVacc: 0.6164957849168375
-------------Gamma e C---------------: 85.31678524172814 0.0030391953823131978
CVacc: 0.6164957849168375
-------------Gamma e C---------------: 188.73918221350996 0.0030391953823131978
CVacc: 0.6164957849168375
-------------Gamma e C---------------: 417.5318936560409 0.0030391953823131978
CVacc: 0.6164957849168375
-------------Gamma e C---------------: 923.6708571873884 0.0030391953823131978
CVacc: 0.6164957849168375
-------------Gamma e C---------------: 2043.359717856948 0.0030391953823131978
CVacc: 0.6164957849168375
-------------Gamma e C---------------: 4520.35365636024 0.0030391953823131978
CVacc: 0.6164957849168375
-------------Gamma e C---------------: 10000.0 0.0030391953823131978
CVacc: 0.6164957849168375
-------------Gamma e C---------------: 1e-06 0.005298316906283708
CVacc: 0.6164957849168375
-------------Gamma e C---------------: 2.21221629107

CVacc: 0.6164957849168375
-------------Gamma e C---------------: 0.0028072162039411786 0.01610262027560939
CVacc: 0.6164957849168375
-------------Gamma e C---------------: 0.006210169418915616 0.01610262027560939
CVacc: 0.6164957849168375
-------------Gamma e C---------------: 0.013738237958832637 0.01610262027560939
CVacc: 0.6164957849168375
-------------Gamma e C---------------: 0.03039195382313201 0.01610262027560939
CVacc: 0.6164957849168375
-------------Gamma e C---------------: 0.06723357536499334 0.01610262027560939
CVacc: 0.6164957849168375
-------------Gamma e C---------------: 0.14873521072935117 0.01610262027560939
CVacc: 0.6164957849168375
-------------Gamma e C---------------: 0.3290344562312671 0.01610262027560939
CVacc: 0.6164957849168375
-------------Gamma e C---------------: 0.7278953843983161 0.01610262027560939
CVacc: 0.6164957849168375
-------------Gamma e C---------------: 1.6102620275609425 0.01610262027560939
CVacc: 0.6164957849168375
-------------Gamma e C------

CVacc: 0.6164957849168375
-------------Gamma e C---------------: 10000.0 0.04893900918477494
CVacc: 0.6164957849168375
-------------Gamma e C---------------: 1e-06 0.0853167852417281
CVacc: 0.6164957849168375
-------------Gamma e C---------------: 2.2122162910704504e-06 0.0853167852417281
CVacc: 0.6164957849168375
-------------Gamma e C---------------: 4.893900918477499e-06 0.0853167852417281
CVacc: 0.6164957849168375
-------------Gamma e C---------------: 1.082636733874054e-05 0.0853167852417281
CVacc: 0.6164957849168375
-------------Gamma e C---------------: 2.395026619987486e-05 0.0853167852417281
CVacc: 0.6164957849168375
-------------Gamma e C---------------: 5.2983169062837125e-05 0.0853167852417281
CVacc: 0.6164957849168375
-------------Gamma e C---------------: 0.00011721022975334806 0.0853167852417281
CVacc: 0.6164957849168375
-------------Gamma e C---------------: 0.0002592943797404667 0.0853167852417281
CVacc: 0.6164957849168375
-------------Gamma e C---------------: 0.00057

CVacc: 0.6164957849168375
-------------Gamma e C---------------: 1.6102620275609425 0.2592943797404667
CVacc: 0.6164957849168375
-------------Gamma e C---------------: 3.562247890262444 0.2592943797404667
CVacc: 0.6164957849168375
-------------Gamma e C---------------: 7.880462815669921 0.2592943797404667
CVacc: 0.6179824561403509
-------------Gamma e C---------------: 17.43328822199991 0.2592943797404667
CVacc: 0.6805080884028253
-------------Gamma e C---------------: 38.56620421163472 0.2592943797404667
CVacc: 0.7061289587605376
-------------Gamma e C---------------: 85.31678524172814 0.2592943797404667
CVacc: 0.7799840510366826
-------------Gamma e C---------------: 188.73918221350996 0.2592943797404667
CVacc: 0.8244588744588743
-------------Gamma e C---------------: 417.5318936560409 0.2592943797404667
CVacc: 0.8606402369560262
-------------Gamma e C---------------: 923.6708571873884 0.2592943797404667
CVacc: 0.8531271360218728
-------------Gamma e C---------------: 2043.3597178569

...

CVacc: 0.6488493962178173
-------------Gamma e C---------------: 1.6102620275609425 22.122162910704503
CVacc: 0.6721975393028025
-------------Gamma e C---------------: 3.562247890262444 22.122162910704503
CVacc: 0.7023980405559352
-------------Gamma e C---------------: 7.880462815669921 22.122162910704503
CVacc: 0.7129129642287536
-------------Gamma e C---------------: 17.43328822199991 22.122162910704503
CVacc: 0.7935748462064252
-------------Gamma e C---------------: 38.56620421163472 22.122162910704503
CVacc: 0.8553599908863068
-------------Gamma e C---------------: 85.31678524172814 22.122162910704503
CVacc: 0.8907325131009343
-------------Gamma e C---------------: 188.73918221350996 22.122162910704503
CVacc: 0.9163647755753018
-------------Gamma e C---------------: 417.5318936560409 22.122162910704503
CVacc: 0.9186375028480289
-------------Gamma e C---------------: 923.6708571873884 22.122162910704503
CVacc: 0.920146958304853
-------------Gamma e C---------------: 2043.35971785694

CVacc: 0.6164957849168375
-------------Gamma e C---------------: 0.0012689610031679235 117.21022975334817
CVacc: 0.6164957849168375
-------------Gamma e C---------------: 0.0028072162039411786 117.21022975334817
CVacc: 0.6164957849168375
-------------Gamma e C---------------: 0.006210169418915616 117.21022975334817
CVacc: 0.6164957849168375
-------------Gamma e C---------------: 0.013738237958832637 117.21022975334817
CVacc: 0.6164957849168375
-------------Gamma e C---------------: 0.03039195382313201 117.21022975334817
CVacc: 0.6164957849168375
-------------Gamma e C---------------: 0.06723357536499334 117.21022975334817
CVacc: 0.6164957849168375
-------------Gamma e C---------------: 0.14873521072935117 117.21022975334817
CVacc: 0.6164957849168375
-------------Gamma e C---------------: 0.3290344562312671 117.21022975334817
CVacc: 0.6488493962178173
-------------Gamma e C---------------: 0.7278953843983161 117.21022975334817
CVacc: 0.6676691729323309
-------------Gamma e C------------

CVacc: 0.8862269309637729
-------------Gamma e C---------------: 10000.0 356.2247890262444
CVacc: 0.8048758259284576
-------------Gamma e C---------------: 1e-06 621.0169418915616
CVacc: 0.6164957849168375
-------------Gamma e C---------------: 2.2122162910704504e-06 621.0169418915616
CVacc: 0.6164957849168375
-------------Gamma e C---------------: 4.893900918477499e-06 621.0169418915616
CVacc: 0.6164957849168375
-------------Gamma e C---------------: 1.082636733874054e-05 621.0169418915616
CVacc: 0.6164957849168375
-------------Gamma e C---------------: 2.395026619987486e-05 621.0169418915616
CVacc: 0.6164957849168375
-------------Gamma e C---------------: 5.2983169062837125e-05 621.0169418915616
CVacc: 0.6164957849168375
-------------Gamma e C---------------: 0.00011721022975334806 621.0169418915616
CVacc: 0.6164957849168375
-------------Gamma e C---------------: 0.0002592943797404667 621.0169418915616
CVacc: 0.6164957849168375
-------------Gamma e C---------------: 0.000573615251044

CVacc: 0.7249658236500343
-------------Gamma e C---------------: 3.562247890262444 1887.3918221350996
CVacc: 0.7453691045796309
-------------Gamma e C---------------: 7.880462815669921 1887.3918221350996
CVacc: 0.8425666438824333
-------------Gamma e C---------------: 17.43328822199991 1887.3918221350996
CVacc: 0.8794372294372295
-------------Gamma e C---------------: 38.56620421163472 1887.3918221350996
CVacc: 0.9186147186147187
-------------Gamma e C---------------: 85.31678524172814 1887.3918221350996
CVacc: 0.9088459785828207
-------------Gamma e C---------------: 188.73918221350996 1887.3918221350996
CVacc: 0.9005753018910911
-------------Gamma e C---------------: 417.5318936560409 1887.3918221350996
CVacc: 0.9050979722032354
-------------Gamma e C---------------: 923.6708571873884 1887.3918221350996
CVacc: 0.9163704716336295
-------------Gamma e C---------------: 2043.359717856948 1887.3918221350996
CVacc: 0.9141091364775575
-------------Gamma e C---------------: 4520.35365636024

CVacc: 0.6164957849168375
-------------Gamma e C---------------: 0.006210169418915616 10000.0
CVacc: 0.6164957849168375
-------------Gamma e C---------------: 0.013738237958832637 10000.0
CVacc: 0.6164957849168375
-------------Gamma e C---------------: 0.03039195382313201 10000.0
CVacc: 0.6465823650034176
-------------Gamma e C---------------: 0.06723357536499334 10000.0
CVacc: 0.6601560719981774
-------------Gamma e C---------------: 0.14873521072935117 10000.0
CVacc: 0.6895705172020962
-------------Gamma e C---------------: 0.3290344562312671 10000.0
CVacc: 0.7016404647983595
-------------Gamma e C---------------: 0.7278953843983161 10000.0
CVacc: 0.7091592617908408
-------------Gamma e C---------------: 1.6102620275609425 10000.0
CVacc: 0.7377762588288904
-------------Gamma e C---------------: 3.562247890262444 10000.0
CVacc: 0.8131578947368421
-------------Gamma e C---------------: 7.880462815669921 10000.0
CVacc: 0.8726817042606516
-------------Gamma e C---------------: 17.4332882

In [ ]:
#tool for testing of frank with increasing train and fix test sampled fair 50%50%
#NO new model selection
cvacc=0
nfold=100
kf = KFold(n_splits=nfold, random_state=13, shuffle=True)
bestc=3290.344562312671
bestgamma=85.31678524172814
x_tvr=[]
y_tvr=[]
end=0
stepsize=100
nstep=len(x_tv)//stepsize
accvector=[]
niter=10
for i in range(1,nstep):
    x_tvr=x_tv[0:i*stepsize]
    y_tvr=y_tv[0:i*stepsize]
    cvacc=0
    print(i)
    print(len(x_tvr))
    for k in range(0,niter):
        print('----',k)
        rbf_svm=svm.SVC(kernel='rbf',C=bestc,gamma=bestgamma)
        rbf_svm.fit(x_tvr,y_tvr)
        accuracy=rbf_svm.score(x_testnew,y_testnew)
        cvacc=cvacc+accuracy/niter
        
    accvector.append(cvacc)
#VETTORE FINALE
print('Final accuracy vector along time')
print(accvector)
        
    
    
    

In [ ]:
#tool for testing of frank with increasing train and fix test sampled fair 50%50%
#YES new model selection this means that every step time i have to redo the model selection
cvacc=0
nfold=10
kf = KFold(n_splits=nfold, random_state=13, shuffle=True)
bestc=0.7278953843983161
bestgamma=17.43328822199991
x_tvr=[]
y_tvr=[]
end=0
stepsize=10
nstep=len(x_tv)//stepsize
accvector=[]
for i in range(1,nstep):
    x_tvr=x_tv[1:i*stepsize]
    y_tvr=y_tv[1:i*stepsize]
    cvacc=0
    for train_index, test_index in kf.split(x_tvr):
            #print("TRAIN:", train_index, "TEST:", test_index)
            x_train, x_val = x_tvr[train_index], x_tvr[test_index]
            y_train, y_val = y_tvr[train_index], y_tvr[test_index]
            rbf_svm=svm.SVC(kernel='rbf',C=bestc,gamma=bestgamma)
            rbf_svm.fit(x_train,y_train)
            accuracy=rbf_svm.score(x_val,y_val)
            cvacc=cvacc+accuracy/nfold
    accvector.append(cvacc)
print(accvector)
        
    
    
    

In [ ]:
#baseline construction
#cV BASELINE
tot=0
for i in range(1,len(ytrain)):
    if(ytrain[i]==1):
        tot+=1
baseacc=tot/len(ytrain)
print('Baseline CV accuracy:',baseacc)
#test BASELINE
tot=0
for i in range(1,len(ytest)):
    if(ytest[i]==1):
        tot+=1
baseacc=tot/len(ytest)
print('Baseline test accuracy:',baseacc)

In [ ]:

#X = np.array([[1, 2], [3, 4], [1, 2], [3, 4],[1, 2], [3, 4], [1, 2], [3, 4],[1, 2], [3, 4], [1, 2], [3, 4],[1, 2], [3, 4], [1, 2], [3, 4]])
#y = np.array([1, 2, 3, 4,1, 2, 3, 4,1, 2, 3, 4,1, 2, 3, 4])
nfold=10
kf = KFold(n_splits=nfold, random_state=None, shuffle=True)
kf.get_n_splits(xtrain)

cspace=np.logspace(-6,4,30)
gspace=np.logspace(-6,4,30)
bestsvm=None
maxacc=0
cvacc=0
maxg=0
maxc=0

for c in cspace:
    
    for g in gspace:
        print('-------------Gamma e C---------------:',g,c)
        cvacc=0
        #crossvalidation
        
        for train_index, test_index in kf.split(xtrain):
            #print("TRAIN:", train_index, "TEST:", test_index)
            x_train, x_test = xtrain[train_index], xtrain[test_index]
            y_train, y_test = ytrain[train_index], ytrain[test_index]
            rbf_svm=svm.SVC(kernel='rbf',C=c,gamma=g)
            rbf_svm.fit(x_train,y_train)
            tot=0
            accuracy=rbf_svm.score(x_test,y_test)
            print(accuracy)
            cvacc=cvacc+accuracy/nfold
        print('CVacc:',cvacc)
        if(cvacc>maxacc):
            maxacc=cvacc
            maxg=g
            maxc=c
            bestsvm=rbf_svm
print('ACCURACY:',maxacc)
print('Gamma:',maxg)
print('C:',maxc)
print(bestsvm)
tot=0

for i in range(1,len(xtest)):
    pred=np.sign(ytest[i]*bestsvm.predict([xtest[i]]))
    if(pred>0):
        tot+=1
accuracy=tot/len(xtest)
testacc=bestsvm.score(xtest,ytest)
print(accuracy)
print('Test ACCURACY ',testacc)


In [ ]:
print(bestsvm)
tot=0
for i in range(1,len(x_test)):

    pred=np.sign(y_test[i]*bestsvm.predict([x_test[i]]))
    if(pred>=0):
        tot+=1
accuracy=tot/len(x_test)
print(accuracy)
testacc=bestsvm.score(x_test,y_test)
print(testacc)


In [ ]:
#Averaging on test accuracy
nk=10
totalacc=[]
for k in range(1,nk):
    print('====================================================================')
    print('nk =',k)
    nfold=10
    kf = KFold(n_splits=nfold, random_state=None, shuffle=True)
    kf.get_n_splits(xtrain)

    cspace=np.logspace(-6,4,30)
    gspace=np.logspace(-6,4,30)
    bestsvm=None
    maxacc=0
    cvacc=0
    maxg=0
    maxc=0
    for c in cspace:
        for g in gspace:
            #print('-------------Gamma e C---------------:',g,c)
            cvacc=0
            #crossvalidation
            for train_index, test_index in kf.split(xtrain):
                #print("TRAIN:", train_index, "TEST:", test_index)
                x_train, x_test = xtrain[train_index], xtrain[test_index]
                y_train, y_test = ytrain[train_index], ytrain[test_index]
                rbf_svm=svm.SVC(kernel='rbf',C=c,gamma=g)
                rbf_svm.fit(x_train,y_train)
                tot=0
                for i in range(1,len(x_test)):

                    pred=np.sign(y_test[i]*rbf_svm.predict([x_test[i]]))
                    if(pred>0):
                        tot+=1
                accuracy=tot/len(x_test)
                cvacc=cvacc+accuracy/nfold
            #print(cvacc)
            if(cvacc>maxacc):
                maxacc=accuracy
                maxg=g
                maxc=c
                bestsvm=rbf_svm
    print('CV ACCURACY:',maxacc)
    print('Gamma:',maxg)
    print('C:',maxc)
    print(bestsvm)
    testacc=bestsvm.score(x_test,y_test)
    
    print('Test ACCURACY ',testacc)
    totalacc.append(testacc)
print('IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII')
print('IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII')
avgtotacc=sum(totalacc)/len(totalacc)
print(totalacc)
print('FINAL MEAN ACCURACY ON TEST:',avgtotacc)

In [ ]:
train=0.7
nt=math.ceil(len(x)*train)
x_train=[]
y_train=[]
x_test=[]
y_test=[]
x_train=x[:nt]
y_train=y[:nt]
x_test=x[nt:]
y_test=y[nt:]


In [ ]:
cspace=np.logspace(-6,4,30)
gspace=np.logspace(-6,4,30)
maxacc=0
for c in cspace:
    for g in gspace:
        print('-------------Gamma e C---------------:',g,c)
        rbf_svm=svm.SVC(kernel='rbf',C=c,gamma=g)
        rbf_svm.fit(x_train,y_train)
        
        tot=0
        for i in range(1,len(x_test)):

            pred=np.sign(y_test[i]*rbf_svm.predict([x_test[i]]))
            if(pred>0):
                tot+=1
        accuracy=tot/len(x_test)
        if(accuracy>maxacc):
            maxacc=accuracy
print(accuracy)

In [ ]:
print(maxacc)

In [ ]:
rbf_svm=svm.SVC(kernel='linear')
rbf_svm.fit(x_train,y_train)



In [ ]:
tot=0
for i in range(1,len(x_test)):
    
    pred=np.sign(y_test[i]*rbf_svm.predict([x_test[i]]))
    print(x_test[i])
    print(rbf_svm.predict([x_test[i]]))
    if(pred>0):
        tot+=1
accuracy=tot/len(x_test)
print(accuracy)